# Modify metadata - remove 'None' metadata values and add empty ones

## Set up

In [ ]:
import os
from irods.session import iRODSSession

cred_path = "D:/iRods_credentials.txt" #file containing iRODS credentials
irods_home = "/tempZone/home/garys" #iRODS home collection
sourceTree = "Y:/garys" #same iRODS home collection, but mapped as a disc drive with davrods

## Get an iRODS session

In [ ]:
creds = [line.rstrip('\n') for line in open(cred_path, "r")]

with iRODSSession(host=creds[0], port=creds[1], user=creds[2], password=creds[3], zone=creds[4], \
                  #client_user=creds[5], \
                  client_server_negotiation = "request_server_negotiation", client_server_policy = "CS_NEG_REQUIRE", \
                  encryption_algorithm = "AES-256-CBC", encryption_key_size = 32, encryption_num_hash_rounds = 16, \
                  encryption_salt_size = 8) as session:
    print ('Got A Session', session)
    pass

collection = session.collections.get(irods_home)
print(collection)

## Loop over Collections

In [ ]:
metadata_file = open("D:\metadataNone.txt", "w")
for sourceDir, subdirList, fileList in os.walk(sourceTree):
    rodsDir = irods_home+sourceDir[len(sourceTree):]
    rodsDir = rodsDir.replace('\\', '/')
    #print(rodsDir)
    if session.collections.exists(rodsDir):
        obj = session.collections.get(rodsDir)
        for item in obj.metadata.items():
            if item.value.strip() == "None":
                metadata_file.write("imeta rmw -C "+rodsDir+" "+item.name.strip()+" % %\n")
            if item.units == '\n':
            #if item.units != None:
                print(repr(item.name), repr(item.value), repr(item.units))
                metadata_file.write("imeta rmw -C "+rodsDir+" "+item.name.strip()+" % % \n")
                metadata_file.write("imeta add -C "+rodsDir+" '"+item.name.strip()+"' '"+item.value+"' '' \n")
metadata_file.close()